In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lumnisai
from lumnisai import Scope, ApiProvider, AsyncClient
from IPython.display import Markdown
from tqdm import tqdm
import asyncio
from dotenv import load_dotenv
import os
from typing import Optional
import json
from pydantic import BaseModel, Field
from lumnisai.types import ModelType


load_dotenv()


def display_markdown(text):
    display(Markdown(text))

# Create client
lumnisai_agent = lumnisai.AsyncClient()

# List users
users = await lumnisai_agent.list_users(page_size=50)

# Check if alice@test-acme.one exists in users
alice_user = next((user for user in users.users if user.email == "alice@test-acme.one"), None)

# Create user if not found
if alice_user is None:
    user = await lumnisai_agent.create_user( email="alice@test-acme.one", first_name="Alice", last_name="Doe" )

# Print users
print("Existing Users:")
for user in users.users:
    print("    ", user.email)

print("="*50)

# List API keys
api_keys = await lumnisai_agent.list_api_keys()
print("Existing API keys:")
for api_key in api_keys:
    print("    ", api_key.provider, "Exists - Created at", api_key.created_at)

print("="*50)

# Add API keys
await lumnisai_agent.add_api_key(ApiProvider.OPENAI_API_KEY, os.getenv("OPENAI_API_KEY"))
await lumnisai_agent.add_api_key(ApiProvider.EXA_API_KEY, os.getenv("EXA_API_KEY"))

preferences = await lumnisai_agent.get_model_preferences()
print(preferences)

print("Setup Complete!")

Existing Users:
     kevin@ksylvest.com
     ari@sisu.site
     amboufaye1@gmail.com
     vyahhi@gmail.com
     ajaye2@illinois.edu
     abubakarr_jaye@alumni.brown.edu
     lumnistrading@gmail.com
     moribajaye@gmail.com
     abujaye@gmail.com
     alice@test-acme.one
     abu.jaye@gmail.com
     hamza@test-acme.one
Existing API keys:
     EXA_API_KEY Exists - Created at 2025-06-20T02:50:30.419402+00:00
     OPENAI_API_KEY Exists - Created at 2025-06-20T02:50:25.676567+00:00
Model Preferences (Tenant: fa12c8b3-6f5d-434c-9797-fdbe8c1f6915):
  CHEAP_MODEL: openai:gpt-4.1-nano ✓
  FAST_MODEL: openai:gpt-4.1-mini ✓
  SMART_MODEL: openai:gpt-4.1 ✓
  REASONING_MODEL: openai:o3-mini ✓
  VISION_MODEL: openai:gpt-4o ✓

Defaults applied for: CHEAP_MODEL
Setup Complete!


In [3]:
new_preferences = {
    "FAST_MODEL": {
        "provider": "openai",
        "model_name": "gpt-4.1-mini"
    },
    "SMART_MODEL": {
        "provider": "openai",
        "model_name": "gpt-4.1"
    },
    "REASONING_MODEL": {
        "provider": "openai",
        "model_name": "o3-mini"
    }
}

updated = await lumnisai_agent.update_model_preferences(new_preferences)
print(updated)

Model Preferences (Tenant: fa12c8b3-6f5d-434c-9797-fdbe8c1f6915):
  CHEAP_MODEL: openai:gpt-4.1-nano ✓
  FAST_MODEL: openai:gpt-4.1-mini ✓
  SMART_MODEL: openai:gpt-4.1 ✓
  REASONING_MODEL: openai:o3-mini ✓
  VISION_MODEL: openai:gpt-4o ✓

Defaults applied for: CHEAP_MODEL


In [4]:
user_email = "abu.jaye@gmail.com" # "alice@test-acme.one"

In [5]:
# List apps
apps = await lumnisai_agent.list_apps(include_available=True)
print("These are all the apps that are available to you:")
print(apps)

# Enable apps to tenant if not enabled
print("="*50)
apps_to_connect = ["github", "gmail", ]#'googledocs' 
for app in apps_to_connect:
    if not await lumnisai_agent.is_app_enabled(app):
        print(f"   {app} is not enabled. Enabling it now...")
        result = await lumnisai_agent.set_app_enabled(app, enabled=True)
    else:
        print(f"   {app} is already enabled.")


for app in apps_to_connect:
    print(f"Checking {app}...")
    if await lumnisai_agent.is_app_enabled(app):
        status = await lumnisai_agent.get_connection_status( user_id=user_email, app_name=app)
        if status.status != "active":
            print(f"   {app} is enabled. Connecting to it now...")
            connection = await lumnisai_agent.initiate_connection( user_id=user_email, app_name=app)
            print("Sign in using the following URL to connect to the app: ", connection)

for app in apps_to_connect:
    print(f"Checking {app}...")
    status = await lumnisai_agent.get_connection_status(user_id=user_email, app_name=app)
    print(f"   App: {status.app_name}")
    print(f"   Status: {status.status}")
    if status.connected_at:
        print(f"   Connected at: {status.connected_at}")

These are all the apps that are available to you:
App Status:
  Enabled Apps (3): GITHUB, GMAIL, SLACK
  Available Apps (144): BRIGHTPEARL, SLACKBOT, YANDEX, SALESFORCE, ATTIO, SUPABASE, AIRTABLE, LEVER, TWITCH, FIGMA, TODOIST, CLOCKIFY, TWITTER, GITLAB, BORNEO, WRIKE, BRAINTREE, BREX_STAGING, ZOHO_INVENTORY, FITBIT, RIPPLING, SURVEY_MONKEY, D2LBRIGHTSPACE, GO_TO_WEBINAR, INTERCOM, CONFLUENCE, LEVER_SANDBOX, BITBUCKET, ASANA, GOOGLEADS, SEISMIC, CANVAS, SHOPIFY, TIMECAMP, CLOSE, CHATWORK, JIRA, SQUARE, ACCELO, CANVA, FACEBOOK, GOOGLESHEETS, ZOHO, REDDIT, KOMMO, GITHUB, BOX, ALGOLIA, ZOHO_INVOICE, BREVO, CLICKUP, MURAL, ZOOM, BOLDSIGN, ZOHO_MAIL, BATTLENET, HUBSPOT, GOOGLETASKS, GOOGLE_MAPS, RAMP, TRELLO, GUMROAD, AMAZON, DROPBOX, MAILCHIMP, METAADS, BREX, LODGIFY, DYNAMICS365, GOOGLESLIDES, LINKEDIN, BLACKBAUD, TONEDEN, WAKATIME, CONTENTFUL, WAVE_ACCOUNTING, KEAP, ZOHO_BIGIN, CALENDLY, ICIMS_TALENT_CLOUD, SLACK, PIPEDRIVE, SPLITWISE, XERO, GOOGLE_ANALYTICS, NETSUITE, EXCEL, OUTLOOK, IN

In [6]:
# List connections
print("="*50)
connections = await lumnisai_agent.list_connections(user_email)
print(connections)

# List tools
print("="*50)
tools = await lumnisai_agent.get_integration_tools(user_email)
print(tools)

User Connections (User ID: abu.jaye@gmail.com):
  GMAIL: active (connected: 2025-07-03 21:54:39.264666)
  GITHUB: active (connected: 2025-07-02 00:08:03.948789)

Total connections: 2
Available Tools (User ID: abu.jaye@gmail.com):
  GMAIL:
    • GMAIL_DELETE_DRAFT: Permanently deletes a specific gmail draft using its id; ensure the draft exists and the user has necessary permissions for the given `user id`.
    • GMAIL_DELETE_MESSAGE: Permanently deletes a specific email message by its id from a gmail mailbox; for `user id`, use 'me' for the authenticated user or an email address to which the authenticated user has delegated access.
    • GMAIL_FETCH_EMAILS: Fetches a list of email messages from a gmail account, supporting filtering, pagination, and optional full content retrieval.
    • GMAIL_FETCH_MESSAGE_BY_MESSAGE_ID: Fetches a specific email message by its id, provided the `message id` exists and is accessible to the authenticated `user id`.
    • GMAIL_GET_ATTACHMENT: Retrieves a 

In [ ]:

# Streaming response  
async for update in await lumnisai_agent.invoke(""" 
                                                What are the latest trends in AI agents and machine learning? 
                                                What are agentic models? 
                                                What are the latest papers on agentic models and agentic AI?. 
                                                """, stream=True, user_id=user_email):
    print(f"{update.state.upper()} - {update.message}")

display(Markdown(update.output_text))


Response ID: 6eee8134-6f81-4017-836a-94de2a7005f2
PROCESSING - Agent thinking...
PROCESSING - Researching and summarizing the most recent (2025) trends in AI agents and machine learning.
PROCESSING - Explaining what 'agentic models' are, key characteristics, and how they differ from traditional models.
PROCESSING - Compiling a list of the most recent (2024-2025) research papers on agentic models and agentic AI, with links, abstracts, and publication dates.


In [ ]:
"""
Example demonstrating how to use structured output with the Lumnis SDK.

The SDK supports two ways to specify response format:
1. Pass a Pydantic model class directly (recommended)
2. Pass a JSON Schema dictionary
"""
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: Optional[str] = None

class BusinessInfo(BaseModel):
    name: str = Field(description="Business name")
    category: str = Field(description="Type of business")
    address: Address
    phone: Optional[str] = None
    website: Optional[str] = None
    rating: Optional[float] = Field(None, ge=0, le=5)


# Example: Complex structure with nested models
response_structured = await lumnisai_agent.invoke(
    messages=[{"role": "user", "content": "Tell me about the Louvre Museum"}],
    response_format=BusinessInfo,  # Pass the model class directly
    response_format_instructions="Include all available details",
    user_id=user_email
)

if response_structured.structured_response:
    museum = BusinessInfo(**response_structured.structured_response)
    print(f"Museum: {museum.name}")
    print(f"Location: {museum.address.city}, {museum.address.country}")

    print(json.dumps(response_structured.structured_response, indent=4))



Response ID: 536f5f5c-aeed-4469-80ca-e8d0ac30a218
Museum: Louvre Museum
Location: Paris, France
{
    "name": "Louvre Museum",
    "address": {
        "city": "Paris",
        "street": "Rue de Rivoli",
        "country": "France",
        "postal_code": "75001"
    },
    "category": "Art Museum"
}


In [ ]:
response_structured.progress

[ProgressEntry(ts=datetime.datetime(2025, 7, 4, 0, 4, 14, 23458, tzinfo=TzInfo(UTC)), state='processing', message='Starting agent...', output_text=None),
 ProgressEntry(ts=datetime.datetime(2025, 7, 4, 0, 4, 28, 436610, tzinfo=TzInfo(UTC)), state='completed', message='Response completed successfully.', output_text=None)]

In [ ]:
# Blocking response (default)
response = await lumnisai_agent.invoke("What are the latest trends in AI agents and machine learning?", user_id=user_email, show_progress=True)
display(Markdown(response.output_text))

Response ID: 766f5d48-2f80-4de5-a687-f53e36b3ff3d
PROCESSING: Starting agent...
PROCESSING: Searching for the most recent (2024-2025) developments, publications, and news on AI agents.
PROCESSING: Searching for the most recent (2024-2025) developments, publications, and news on machine learning.
PROCESSING: Synthesizing and analyzing the latest (2024–2025) trends in AI agents, highlighting architectures, benchmarks, applications, and breakthroughs.
PROCESSING: Synthesizing and analyzing the latest (2024–2025) trends in machine learning, focusing on algorithms, frameworks, applications, and breakthroughs.
PROCESSING: Combining the analysis of 2024–2025 trends in AI agents and machine learning into a clear, structured, well-organized report with non-overlapping coverage.
PROCESSING: Carefully review the unified report on latest trends in AI agents and machine learning for completeness, accuracy, and clarity, revising sections if needed.
PROCESSING: Send the requester the final, reviewed 

# Latest Trends in AI Agents and Machine Learning (2024–2025)

---

## Executive Summary

From early 2024 through mid-2025, artificial intelligence has advanced dramatically in both agent-based systems and foundational machine learning (ML). AI agents have become sophisticated, modular, and context-aware, driving automation and new capabilities across industries. Machine learning breakthroughs underpin these agents with robust, efficient algorithms, transformative architectures, and new training paradigms. Below is a clear synthesis of the non-overlapping developments and key trends in both domains.

---

## 1. Advances in AI Agents

### 1.1 New Architectures and Capabilities

- **LLMs as Agentic Cores:**  
  Models like GPT-4/5, PaLM, and Bedrock now power reasoning, planning, and multi-turn dialogue in modern agents.  
  [IEEE Computer Society](https://www.computer.org/publications/tech-news/community-voices/autonomous-ai-agents)

- **Memory-Augmented and Contextual Agents:**  
  Agents routinely use long-term and episodic memory, supporting richer context, autonomy, and adaptation.  
  [Arxiv](https://arxiv.org/abs/2501.11739)

- **Multi-Agent and Decentralized Systems:**  
  Teams ("swarms") of agents, often decentralized, now tackle complex real-world challenges together.  
  [Forbes](https://www.forbes.com/sites/solrashidi/2025/02/28/the-5-ai-trends-in-2025-agents-open-source-and-multi-model/)

- **Tool Use and Retrieval-Augmented Generation (RAG):**  
  Agents autonomously leverage APIs, databases, and internet search—blending traditional software engineering and generative reasoning.  
  [LinkedIn](https://www.linkedin.com/pulse/inside-ai-agents-how-perceive-decide-act-learn-artemakis-artemiou-1nzrf)

- **Multimodal and Edge-Integrated Agents:**  
  New agents process voice, images, video, and sensor data—using both on-device and cloud resources for scalability.  
  [ScienceDirect](https://www.sciencedirect.com/science/article/pii/S2949953425000141)

---

### 1.2 Benchmarks and Evaluation

- Evaluation now emphasizes long-horizon planning, human–agent collaboration, and memory retention.
- Productivity, satisfaction, and ethical dimensions are increasingly key metrics.  
  [Arxiv](https://arxiv.org/abs/2501.11739)

---

### 1.3 Notable Applications

- **Enterprise Automation:** Document parsing, analytics, workflow orchestration, and support.
- **Healthcare:** Diagnostics, triage, and care simulation.
- **Finance & Smart Cities:** Risk modeling, accessibility, urban/logistics management.
- **Personal Productivity:** Context-aware assistants for scheduling and information.

---

### 1.4 Breakthroughs and Industry Trends

- **Open Source Agent Frameworks:** LangChain, AutoGPT, and Copilot Studio speed agent creation.
- **Custom/Plug-and-Play Agents:** Industry is rapidly adopting tailored, domain-specific agents.
- **Responsible AI & Memory Regulation:** Safe, explainable, and privacy-conscious design is central.

---

## 2. Advances in Machine Learning

### 2.1 New Algorithms and Core Breakthroughs

- **Segment Anything Model 2 (SAM 2):** Real-time video tracking extends from static images to many domains.  
  [MachineLearningMastery.com](https://machinelearningmastery.com/5-breakthrough-machine-learning-research-papers-already-in-2025/)

- **Speculative Cascades:** Cascading between fast and complex models improves inference efficiency.

- **Data Shapley/Attribution:** Efficiently values each training sample, streamlining data quality and compliance.

- **Transformers’ Robustness:** Theoretical advances now clarify why transformers generalize reliably.

- **Continual & Real-Time Learning:** New models like StreamLearn 2.0 adapt on-the-fly for evolving data.

---

### 2.2 Frameworks and Infrastructural Shifts

- **Neuromorphic Hardware & CRAM:** Neuro-inspired chips and in-memory computing enable up to 1,000× energy efficiency.  
  [ScienceDaily](https://www.sciencedaily.com/releases/2024/07/240726113337.htm)

- **Small Language Models (SLMs):** Compact (1–10B param) open models democratize LLMs for edge/mobile use.  
  [Medium](https://medium.com/@taiwo.omotayo/the-cutting-edge-of-machine-learning-whats-new-in-2025-b12fc61dc862)

- **Causal Inference & Explainability:** Widespread adoption of causal methods increases fairness and transparency.

- **Federated & Privacy-Preserving Learning:** Models learn locally, keeping data private—critical for healthcare/IoT.

---

### 2.3 Emerging Trends

- **Meta-Learning & Extrapolation:** E2T and meta-learning enable robust generalization for rare/unseen scenarios.  
  [Phys.org](https://phys.org/news/2025-04-ai-extrapolative-master-materials.html)

- **Human-in-the-Loop Training:** Real-time, interactive feedback (GUIDE framework) is closing the gap to dynamic learning.  
  [ScienceDaily](https://www.sciencedaily.com/releases/2024/12/241203154556.htm)

- **Quantum Machine Learning:** Quantum compilation algorithms now speed up scientific simulation.  
  [ScienceDaily](https://www.sciencedaily.com/releases/2024/12/241210115620.htm)

- **Brain-like Machine Vision:** Architectures such as Lp-Convolution improve adaptability in AI vision.

---

### 2.4 Real-World Applications

- **Robotics:** RL enables flexible, learning robots across tasks and settings.
- **Weather Modeling:** ML delivers 15-day forecasts as accurate as old 7-day predictions.
- **Scientific Discovery & Mathematics:** AI assists proofs, simulation, and research.
- **Healthcare:** Synthetic data, privacy, and advanced segmentation shift diagnostics.

---

## 3. Synthesis and Distinction

- **AI agents** operationalize memory, collaboration, autonomous action, and tool-based problem-solving.
- **Machine learning** advances provide the robust, efficient, interpretable, and scalable foundations that enable these agentic capabilities.
- These trends are distinct but deeply synergistic; agent breakthroughs drive and embody ML frontiers, while ML innovations extend agent power and scope.

---

## References

- [IEEE Computer Society, 2025](https://www.computer.org/publications/tech-news/community-voices/autonomous-ai-agents)
- [Arxiv, 2025](https://arxiv.org/abs/2501.11739)
- [ScienceDaily, 2025](https://www.sciencedaily.com/releases/2025/04/250422131924.htm)
- [Phys.org, 2025](https://phys.org/news/2025-04-ai-extrapolative-master-materials.html)
- [Medium, 2025](https://medium.com/@taiwo.omotayo/the-cutting-edge-of-machine-learning-whats-new-in-2025-b12fc61dc862)
- [MachineLearningMastery.com, 2025](https://machinelearningmastery.com/5-breakthrough-machine-learning-research-papers-already-in-2025/)
- [Forbes, 2025](https://www.forbes.com/sites/solrashidi/2025/02/28/the-5-ai-trends-in-2025-agents-open-source-and-multi-model/)
- [ScienceDirect, 2025](https://www.sciencedirect.com/science/article/pii/S2949953425000141)
- [LinkedIn, 2025](https://www.linkedin.com/pulse/inside-ai-agents-how-perceive-decide-act-learn-artemakis-artemiou-1nzrf)

*Consult the cited links for full technical details or request a deeper supplement on any focus area.*

---

In [ ]:
user_scoped_client = lumnisai.AsyncClient().for_user("alice@test-acme.one")
response = await user_scoped_client.invoke(
    prompt="What is the weather in Tokyo?",
    show_progress=True
)
# print(response.progress)
display_markdown(response.output_text)

Response ID: 734d003d-71ed-4447-b292-d50467f42429
PROCESSING: Starting agent...
COMPLETED: Response completed successfully.


As of the latest report, the current weather in Tokyo, Japan is:

- **Temperature:** 56°F (13°C)
- **Conditions:** Sunny
- **Humidity:** 73%
- **Wind:** Light, with gusts up to 4 mph
- **Visibility:** 7 miles
- **Dew Point:** 43°F
- **Atmospheric Pressure:** 29.98 inHg

Today, the high is forecasted to reach 61°F, with generally sunny weather and a few clouds.

For updated details, visit: [Weather Underground - Tokyo, Japan](https://www.wunderground.com/weather/jp/tokyo)  
_Source: Weather Underground, updated June 28, 2025_

In [ ]:
response = await user_scoped_client.invoke(
    prompt="What are the latest trends in AI agents and machine learning? ",
    show_progress=True
    )
# print(response.progress)
display_markdown(response.output_text)

Response ID: 9f8f8bb6-6588-4e11-b874-0c9da6e8773e
PROCESSING: Starting agent...
PROCESSING: Research the most recent (2024-2025) trends in AI agents, referencing reputable sources like academic papers, blogs, and news.
PROCESSING: Research the most recent (2024-2025) trends in machine learning, providing relevant references from academic and reputable industry sources.
PROCESSING: Analyze and synthesize the latest trends in AI agents and machine learning (2024-2025), highlighting overlaps, unique developments, and emerging directions, with specific references.
PROCESSING: Write a comprehensive report (with executive summary, detailed trends, synthesis, and references) on the latest (2024-2025) trends in AI agents and machine learning.
PROCESSING: Send a notification message to stakeholders to inform them the AI agents and machine learning trends report is finished, including the report content or a link to it.
COMPLETED: Response completed successfully.


# Report: Latest Trends in AI Agents and Machine Learning (2024–2025)

---

## Executive Summary

AI agents and machine learning (ML) are converging to power autonomous, adaptable, and collaborative digital intelligence. In 2025, AI agents are widely deployed across industries, leveraging breakthroughs in multimodal, generative models. ML fuels this ecosystem with scalable, explainable, and privacy-preserving approaches. Key trends include explosive adoption, advances in autonomy and multimodality, democratization via no-code/low-code tools, human-centered design, and the active emergence of standards for reliability and governance.

---

## 1. AI Agents: Key Trends
- **Mainstream Deployment:** Over 99% of enterprise developers use or explore AI agents across core business functions (finance, healthcare, supply chain, customer service). Market projected to reach $7.92B in 2025 and $236B by 2034.  
  *[IBM, Precedence Research, Master of Code]*
- **Complex, Autonomous, Multi-Agent Systems:** Agents now coordinate with each other using new agent-to-agent protocols (A2A, MCP) and orchestrate multifaceted workflows.  
  *[Capgemini, MIT TechReview]*
- **Next-Gen Platforms:** Major tech players have released agent SDKs and marketplaces (Microsoft Agent Store, Google Gemini Console, AWS, Salesforce, Nvidia).  
  *[CRN, Daily AI Agent News]*
- **Multimodal & Embodied Agents:** Agents process and act on text, vision, speech, and physical environments (e.g., GPT-4o, Gemini, Nvidia Eureka).  
  *[MIT TechReview, Medium]*
- **Human-Augmentation:** Copilot-style agents thrive, especially where safety, compliance, or regulation is key.  
  *[IBM]*
- **Challenges:** Reliability, security, privacy, and energy demands are leading concerns.  
  *[LangChain Survey, MIT TechReview, Deloitte]*

---

## 2. Machine Learning: Key Trends
- **Generative & Multimodal Models:** Foundation models (GPT-4o, Gemini, LLaMA) integrate text, image, video, and audio, boosting agentic and general ML use-cases.  
  *[Machine Learning Mastery, LinkedIn]*
- **Autonomous/Agentic AI:** ML drives ever more capable software agents via advances in reinforcement learning, large context, and self-supervision.  
  *[Machine Learning Mastery, Medium]*
- **Explainable, Ethical, and Responsible AI:** Growing demand for fairness, transparency, and compliance in ML, supported by regulatory moves in healthcare, finance, and law.  
  *[GeeksforGeeks, Medium]*
- **Federated & Edge ML:** Privacy-preserving, decentralized deployment of ML models, vital for sensitive industries like healthcare, IoT, and finance.  
  *[Medium, GeeksforGeeks]*
- **Data-Centric AI:** Synthetic datasets, AutoML tools, and low-code/no-code platforms lower barriers for non-experts.  
  *[Medium, LinkedIn]*
- **Quantum and Causal ML:** Early-stage hybrid quantum-classical and causal inference techniques are emerging.  
  *[LinkedIn, Medium]*
- **Real-Time/Continual Learning:** Models adapt in real-time from streaming data, powering dynamic business and industrial apps.  
  *[Medium, LinkedIn]*

---

## 3. Synthesis & Comparative Analysis

### Overlapping Trends
- **Autonomy & Collaboration:** Both fields are rapidly converging on self-directed, context-adaptive, multi-agent architectures.
- **Multimodality & Generativity:** Foundation models empower agents with broad, generative abilities across data types.
- **Democratization:** Low-code/no-code platforms and marketplaces drive use by domain experts and non-coders.
- **Human-centered, Responsible Intelligence:** Emphasis on human oversight, safety, reliability, and auditability.
- **Common Challenges:** Both struggle with output honesty (hallucinations), compliance, privacy, and energy impact.

### Unique Developments
- **AI Agents:** Focus on workflow automation, agent-to-agent protocols, and agent application marketplaces.
- **Machine Learning:** Advances in foundation models (>10T parameters), continual/self-supervised learning, and novel hardware.

### Emerging Directions
- Continual-learning and end-to-end differentiable agent architectures.
- Standardization enabling ML models and agents to interoperate seamlessly.
- "Ethical agent" frameworks for transparency, fairness, and accountability.

---

## Reference List

1. [IBM: AI Agents in 2025—Expectations vs. Reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality)
2. [Precedence Research: AI Agents Market Size Worth USD 236B by 2034](https://www.globenewswire.com/news-release/2025/07/02/3109307/0/en/AI-Agents-Market-Size-Worth-USD-236-03-Billion-by-2034-Fueled-by-Machine-Learning-and-Natural-Language-Processing-Advances.html)
3. [Master of Code: 150+ AI Agent Statistics (July 2025)](https://masterofcode.com/blog/ai-agent-statistics)
4. [Capgemini/CNBC: AI Agent Collaboration in 2025](https://www.cnbc.com/2024/07/22/ai-that-can-talk-with-other-ai-will-launch-in-2025-capgemini-predicts.html)
5. [MIT Technology Review: What are AI agents? (July 2024)](https://www.technologyreview.com/2024/07/05/1094711/what-are-ai-agents/)
6. [CRN: 10 Hottest Agentic AI Tools Of 2025](https://www.crn.com/news/ai/2025/10-hottest-agentic-ai-tools-and-agents-of-2025-so-far)
7. [AI Agent Store: Daily AI Agent News, May 2025](https://aiagentstore.ai/ai-agent-news/2025-may)
8. [Medium (Edwin Lisowski): AI Agents Take the Lead: 2025](https://medium.com/@elisowski/a-list-of-ai-agents-set-to-dominate-in-2025-028f975c5b99)
9. [Deloitte: TMT 2025 Predictions](https://www2.deloitte.com/us/en/insights/industry/technology/technology-media-and-telecom-predictions/2025/autonomous-generative-ai-agents-still-under-development.html)
10. [Machine Learning Mastery - 7 Machine Learning Trends to Watch in 2025](https://machinelearningmastery.com/7-machine-learning-trends-2025/)
...and more listed in the detailed findings section.

---

**For further detail, see referenced academic articles and industry reports. The convergence of AI agents and ML is defining the new era of adaptive, intelligent digital systems across the global economy.**

Retrieved 0 templates
